In [11]:
import pandas as pd
import plotly.express as px
import plotly.io as pio

import os
from pathlib import Path
import numpy as np
import math
import plotly.graph_objects as go
import numpy as np



In [12]:
path = Path(os.getcwd(), 'data', 'raw_counts.csv')
df = pd.read_csv(path)
df = df.T
df = df.reset_index()
new_header = df.iloc[0] #grab the first row for the header
df = df[1:] #take the data less the header row
df.columns = new_header #set the header row as the df header
df = df.rename(columns={'Unnamed: 0':'year'})
df = df.fillna(0).astype('int64')
df = df.set_index('year')

In [13]:
# create array of species
species = df.columns.values
year = df.reset_index()['year'].values
iterables = [year,species]
index = pd.MultiIndex.from_product(iterables, names=["year", "species"])
df = pd.DataFrame(df.stack(), index=index, columns={'count'}).reset_index()

In [14]:
# total butterflies seen
m_df = df.groupby('year').agg('sum')
fig = px.line(m_df)

fig.update_layout(plot_bgcolor='white', font=dict(family='Arial', size=20))


In [15]:

# fig.update_layout(
#     xaxis=dict(
#         showline=True,
#         showgrid=False,
#         showticklabels=True,
#         linecolor='rgb(204, 204, 204)',
#         linewidth=2,
#         ticks='outside',
#         tickfont=dict(
#             family='Arial',
#             size=12,
#             color='rgb(82, 82, 82)',
#         ),
#     ),
#     yaxis=dict(
#         showgrid=False,
#         zeroline=False,
#         showline=True,
#         showticklabels=True,
#     ),
#     autosize=False,
#     margin=dict(
#         autoexpand=False,
#         l=100,
#         r=20,
#         t=110,
#     ),
#     showlegend=False,
#     plot_bgcolor='white'
# )


In [16]:
print('some \n words')

some 
 words


In [29]:
df

,year,species,count
0,2002,Small/Essex Skipper,11
1,2002,Large Skipper,11
2,2002,Brimstone,0
3,2002,Dingy Skipper,0
4,2002,Large White,24
...,...,...,...
583,2022,Ringlet,34
584,2022,Brown Hairstreak,2
585,2022,Purple Emperor,0
586,2022,Clouded Yellow,0


In [67]:
# all butterflies by count

smooth_df = df.set_index(['year']).groupby('species').rolling(3).mean().reset_index().sort_values('count').dropna()


legend_order = smooth_df.sort_values('count', ascending=True).species.values
col_name = ['species' for i in range(len(legend_order))]
legend_dict = dict(zip(legend_order, col_name))

lab1 = m_df['count'].values[0]
lab2 = m_df['count'].values[0]

fig = px.area(smooth_df, x='year', y='count', color='species',
              title='<b>Butterflies on the reserve have increased 228%<b>',
              width=1200,
              height = 1200,
             
)

fig.update_layout(plot_bgcolor='white',
                  font=dict(family='Arial', size=20), titlefont=dict(family='Arial', size=35),
                  legend=dict(traceorder='reversed'),
                 )

path = Path(os.getcwd(),'vis','Yearly_Counts_By_Species.png')
fig.write_image(path)

fig.show()

In [ ]:
df

In [ ]:
df.set_index(['year','species','type']).rolling(2).sum()

In [ ]:
px.line(df.set_index(['year','species','type']).rolling(1).sum().reset_index(), x='year', y='count', color='species')
# px.line(df, x='year', y='count', color='species')

In [ ]:
grassland = ['Small/Essex Skipper','Dingy Skipper','Small Copper', 'Small Heath','Brown Argus','Common Blue','Marbled White','Gatekeeper','Meadow Brown','Ringlet','Clouded Yellow']
generalist = ['Brimstone','Large White','Small White', 'Green Veined White', 'Holly Blue','Red Admiral','Painted Lady', 'Small Tortoiseshell', 'Peacock', 'Comma']
headgerow = ['Orange Tip', 'Brown Hairstreak', 'Large Skipper']
woodland = ['Silver Washed Fritillary', 'Speckled Wood', 'Purple Emperor', 'Purple Hairstreak']

filt = (df.species.isin(woodland))
df[filt]

In [ ]:
# create a list of our conditions
conditions = [
    (df.species.isin(woodland)),
    (df.species.isin(grassland)),
    (df.species.isin(generalist)),
    (df.species.isin(headgerow))
    ]

# create a list of the values we want to assign for each condition
values = ['Woodland', 'Grassland', 'Generalist', 'Hedgerow']

# create a new column and use np.select to assign values to it using our lists as arguments
df['type'] = np.select(conditions, values)

# display updated DataFrame
df


In [ ]:
g_df = df.groupby(['type','year']).agg('sum')
fig = px.line(g_df.reset_index(), x='year', y='count', color='type', title='Yearly Counts by Butterfly Type')

fig.update_layout(plot_bgcolor='white')


In [ ]:
df.type.unique()

In [ ]:
np.vstack((np.arange(2002, 2022),)*4)


In [ ]:
g_df.head(50)

In [ ]:


title = 'Yearly Butterfly Counts By Type'
labels = ['Generalist', 'Grassland','Hedgerow', 'Woodland']
colors = ['rgb(231,138,195)','rgb(102,194,165)', 'rgb(252,154,98)', 'rgb(141,160,203)']

mode_size = np.array([8, 8, 8, 8])
line_size = np.array([2, 2, 2, 2]) * 2

x_data = np.vstack((np.arange(2002, 2023),)*4)

y_data = g_df.unstack().values

fig = go.Figure()

for i in range(0, 4):
    fig.add_trace(go.Scatter(x=x_data[i], y=y_data[i], mode='lines',
        name=labels[i],
        line=dict(color=colors[i], width=line_size[i]),
        connectgaps=True,
    ))

    # endpoints
    fig.add_trace(go.Scatter(
        x=[x_data[i][-1]],
        y=[y_data[i][-1]],
        mode='markers',
        marker=dict(color=colors[i], size=mode_size[i])
    ))

fig.update_layout(
    xaxis=dict(
        showline=True,
        showgrid=False,
        showticklabels=True,
        linecolor='rgb(204, 204, 204)',
        linewidth=2,
        ticks='outside',
        tickfont=dict(
            family='Arial',
            size=20,
            color='rgb(82, 82, 82)',
        ),
    ),
    yaxis=dict(
        showgrid=False,
        zeroline=False,
        showline=True,
        showticklabels=True,
        tickfont=dict(
            family='Arial',
            size=20,
            color='rgb(82, 82, 82)',
        ),
    ),
    autosize=False,
    margin=dict(
        autoexpand=False,
        l=50,
        r=110,
        t=110,
    ),
    showlegend=False,
    plot_bgcolor='white'
)

annotations = []

# Adding labels
for y_trace, label, color in zip(y_data, labels, colors):
    # labeling the right_side of the plot
    annotations.append(dict(xref='paper', x=0.95, y=y_trace[-1],
                                  xanchor='left', yanchor='middle',
                                  text= label + f' +{round(((y_trace[-1] - y_trace[0]) / y_trace[0])*100)}%',
                                  font=dict(family='Arial',
                                            size=20),
                                  showarrow=False))
# Title
annotations.append(dict(xref='paper', yref='paper', x=0.0, y=1.05,
                              xanchor='left', yanchor='bottom',
                              text='<b>Grassland species are up 3x over the past 20 years<b>',
                              font=dict(family='Arial',
                                        size=30,
                                        color='rgb(37,37,37)'),
                              showarrow=False))
# Source
annotations.append(dict(xref='paper', yref='paper', x=0.5, y=-0.1,
                              xanchor='center', yanchor='top',
                              text='Source: Lingfiled Nature Reserve',
                              font=dict(family='Arial',
                                        size=20,
                                        color='rgb(150,150,150)'),
                              showarrow=False))

fig.update_layout(annotations=annotations, width=1200, height=700, )

fig.show()
path = Path(os.getcwd(),'vis','yearly_butterfly_counts_by_type.png')
fig.write_image(path)


In [ ]:
os.getcwd()


In [ ]:
px.colors.qualitative.Plotly